# Utlyste stillinger til Nav med sklearn

Det som gjøres er inspirert av tutorial [https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f](https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f)

Datasett er hentet fra [https://data.norge.no/organisasjoner/arbeids-og-velferdsetaten-nav](https://data.norge.no/organisasjoner/arbeids-og-velferdsetaten-nav)

Det anbefales å kjøre prosjektet i docker på imaget [continuumio/anaconda3](https://hub.docker.com/r/continuumio/anaconda3/) eller følge instruksjonene i repoet. 



Spørsmål tas gjerne imot hos vikfand@gmail.com

In [1]:
# Set up with packages and expected file structure

!pip install wget html
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [2]:
import HTMLParser

parser = HTMLParser.HTMLParser()
parser.unescape('&amp;')

u'&'

In [3]:
import os
import sys
import sklearn
import pandas as pd
import numpy as np
import xml
import re
from time import time
import matplotlib.pyplot as plt
from pprint import pprint
import html


print(sys.version)
data_dir = os.path.join('.', 'data')

2.7.15 |Anaconda, Inc.| (default, May  1 2018, 23:32:55) 
[GCC 7.2.0]


In [4]:
from lib.data import download_data

# Download the data (will not download if it's already downloaded)
download_data(data_dir)

In [5]:
from lib.load_dataset import load_datasets

# Load datasets into a pandas DataFrame from .csv files
df = load_datasets(data_dir, 2015, 2015)

print(df.shape)
df.head()

Loading datasets from year 2015 to 2015
(118096, 24)


stillingsnummer  nav_enhet_kode registrert_dato sistepubl_dato  \
0  101201412000010             101      18.12.2014     02.01.2015   
1  101201412000011             101      19.12.2014     01.02.2015   
2  101201412000012             101      19.12.2014     16.01.2015   
3  101201412000013             101      23.12.2014     23.01.2015   
4  101201412000014             101      23.12.2014     30.01.2015   

   statistikk_aar_mnd  offisiell_statistikk_flagg             stilling_kilde  \
0              201501                           1          Annonsert i media   
1              201501                           1  Overført fra arbeidsgiver   
2              201501                           1  Overført fra arbeidsgiver   
3              201501                           1          Annonsert i media   
4              201501                           1          Annonsert i media   

  arbeidssted_fylkesnummer arbeidssted_fylke arbeidssted_kommunenummer  \
0                       01           Østfold                      0101   
1                       01           Østfold                      0101   
2                       01           Østfold                      0101   
3                       01           Østfold                      0101   
4                       01           Østfold                      0101   

                         ...                          \
0                        ...                           
1                        ...                           
2                        ...                           
3                        ...                           
4                        ...                           

                yrke_grovgruppe yrkeskode                           yrke  \
0                  Undervisning      2342                 Førskolelærere   
1  Serviceyrker og annet arbeid      5413              Fengselsbetjenter   
2  Serviceyrker og annet arbeid      5413              Fengselsbetjenter   
3        Butikk- og salgsarbeid      3322               Selgere (engros)   
4          Ingeniør- og ikt-fag      2152  Sivilingeniører (elektronikk)   

              yrkesbetegnelse virksomhet_organisasjonsnr  \
0               Førskolelærer                  973967592   
1             Fengselsbetjent                  993315060   
2             Fengselsbetjent                  993315060   
3  Key Account Manager (salg)                  886332882   
4       Forsker (elektronikk)                  886332882   

              virksomhet_navn antall_stillinger  \
0         HØGSKOLEN I ØSTFOLD                 1   
1              HALDEN FENGSEL                 1   
2              HALDEN FENGSEL                 1   
3  SMART INNOVATION NORWAY AS                 1   
4  SMART INNOVATION NORWAY AS                 1   

                                     stillingstittel statistikk_periode  \
0        Barnehagelærar / Førstekonsulent (14/04241)             201501   
1  Halden fengsel - ledig fast stilling som verks...             201501   
2  Halden fengsel - ledig fast stilling som verks...             201501   
3                                Key Account Manager             201501   
4                   Senior Reasearchers/Reasearchers             201501   

                                stillingsbeskrivelse  
0  # senter for framandspråk i opplæringa er det ...  
1  Jobbnorge ID: 108617<br><br><p dir=#LTR# align...  
2  Jobbnorge ID: 108301<br><br><p dir=#LTR# align...  
3  # har ambisjoner om å vokse og styrker teamet ...  
4  # is now expanding its leading team with senio...  

[5 rows x 24 columns]

In [6]:
from lib.preprocessing import remove_tags, unescape_html, remove_whitespace

# Select subset of columns
col = [
    'stilling_kilde', 
    'stillingsnummer', 
    'stillingsbeskrivelse', 
    'yrke_grovgruppe',
    'arbeidssted_fylkesnummer',
    'arbeidssted_kommunenummer',
    'virksomhet_navn',
    'arbeidssted_fylke',
]
df = df[col]


# Preprocess text and add some columns
df['stillingsbeskrivelse'] = df['stillingsbeskrivelse']\
    .map(remove_tags)\
    .map(unescape_html)\
    .map(remove_whitespace)
df['is_from_nav'] = df['stilling_kilde'].map(lambda x: x=='Reg av arb.giver på nav.no')
df['from_media'] = df['stilling_kilde'].map(lambda x: x=='Annonsert i media')
df['is_healthcare'] = df['yrke_grovgruppe'].map(lambda x: x=='Helse, pleie og omsorg')
df['is_industrial'] = df['yrke_grovgruppe'].map(lambda x: x=='Industriarbeid')
df['contains_nav'] = df['stillingsbeskrivelse'].map(lambda x: 'nav' in x)

print(df.shape)
df.head()




/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:19: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


(118096, 13)


stilling_kilde  stillingsnummer  \
0          Annonsert i media  101201412000010   
1  Overført fra arbeidsgiver  101201412000011   
2  Overført fra arbeidsgiver  101201412000012   
3          Annonsert i media  101201412000013   
4          Annonsert i media  101201412000014   

                                stillingsbeskrivelse  \
0  # senter for framandspråk i opplæringa er det ...   
1  Jobbnorge ID: 108617# fengsel er det ledig fas...   
2  Jobbnorge ID: 108301# fengsel er det ledig fas...   
3  # har ambisjoner om å vokse og styrker teamet ...   
4  # is now expanding its leading team with senio...   

                yrke_grovgruppe arbeidssted_fylkesnummer  \
0                  Undervisning                       01   
1  Serviceyrker og annet arbeid                       01   
2  Serviceyrker og annet arbeid                       01   
3        Butikk- og salgsarbeid                       01   
4          Ingeniør- og ikt-fag                       01   

  arbeidssted_kommunenummer             virksomhet_navn arbeidssted_fylke  \
0                      0101         HØGSKOLEN I ØSTFOLD           Østfold   
1                      0101              HALDEN FENGSEL           Østfold   
2                      0101              HALDEN FENGSEL           Østfold   
3                      0101  SMART INNOVATION NORWAY AS           Østfold   
4                      0101  SMART INNOVATION NORWAY AS           Østfold   

   is_from_nav  from_media  is_healthcare  is_industrial  contains_nav  
0        False        True          False          False         False  
1        False       False          False          False         False  
2        False       False          False          False         False  
3        False        True          False          False         False  
4        False        True          False          False         False

In [7]:
from sklearn.model_selection import train_test_split

data_subset_size = len(df) # Just use a small dataset when experimenting or it will take too much time
target_column_name = 'yrke_grovgruppe' # Which label to predict
random_seed = 305

# Make training and test sets of the data. 
def get_train_and_test_sets(df, sample_size=None, test_size=0.2, random_seed=305):
    if not sample_size:
        sample_size = len(df) // 10
    data_sample = df.sample(n=sample_size, random_state=random_seed)
    return train_test_split(
        data_sample, 
        data_sample[target_column_name],
        test_size=.2,
        random_state=random_seed
    )

X_train, X_test, y_train, y_test = get_train_and_test_sets(df)
X_train

stilling_kilde   stillingsnummer  \
28849            Annonsert i media   321201503000035   
86773                    Fra Eures  1502201412000056   
79955            Annonsert i media  1219201502000039   
100626           Annonsert i media  1703201503000018   
85658   Reg av arb.giver på nav.no  1432201506000024   
114752           Annonsert i media  1939201501000006   
49825            Annonsert i media   625201505000013   
86183            Annonsert i media  1439201510000006   
109316           Annonsert i media  1870201509000022   
100685           Annonsert i media  1703201505000018   
5                Annonsert i media   101201412000015   
36254   Reg av arb.giver på nav.no   335201510000072   
67100   Reg av arb.giver på nav.no  1130201507000004   
76025   Reg av arb.giver på nav.no  1209201503000022   
9252             Annonsert i media   219201512000040   
60466            Annonsert i media  1001201504000112   
112474           Annonsert i media  1902201511000052   
73596            Annonsert i media  1205201503000039   
75071            Annonsert i media  1209201412000231   
64703            Annonsert i media  1106201502000103   
52311            Annonsert i media   704201510000172   
38097            Annonsert i media   403201506000010   
86783            Annonsert i media  1502201412000069   
95721            Annonsert i media  1603201509000078   
35439            Annonsert i media   335201503000130   
96337    Overført fra arbeidsgiver  1603201511000220   
94819            Annonsert i media  1603201505000122   
87947   Reg av arb.giver på nav.no  1504201501000089   
6679             Annonsert i media   217201505000043   
88606            Annonsert i media  1504201505000106   
...                            ...               ...   
66955            Annonsert i media  1127201511000002   
11610            Annonsert i media   230201506000019   
73537   Reg av arb.giver på nav.no  1205201502000014   
7880             Annonsert i media   219201504000247   
79854            Annonsert i media  1216201505000008   
83174            Annonsert i media  1251201501000012   
99383            Annonsert i media  1640201508000007   
85803            Annonsert i media  1432201509000059   
11611            Annonsert i media   230201506000023   
66700            Annonsert i media  1124201505000004   
98855            Annonsert i media  1634201503000021   
24571   Reg av arb.giver på nav.no   316201504000215   
58680            Annonsert i media   906201506000078   
50101            Annonsert i media   626201504000022   
54590            Annonsert i media   805201501000091   
25069            Annonsert i media   316201505000362   
63201            Annonsert i media  1101201501000014   
26636   Reg av arb.giver på nav.no   316201510000129   
101803  Reg av arb.giver på nav.no  1721201501000010   
98877            Annonsert i media  1634201504000001   
55045   Reg av arb.giver på nav.no   805201508000094   
58102            Annonsert i media   904201511000003   
615     Reg av arb.giver på nav.no   104201505000017   
63518            Annonsert i media  1102201501000039   
72679            Annonsert i media  1203201412000045   
41199   Reg av arb.giver på nav.no   438201504000013   
100916           Annonsert i media  1703201512000034   
73584            Annonsert i media  1205201503000027   
55949   Reg av arb.giver på nav.no   806201510000005   
3759    Reg av arb.giver på nav.no   124201511000045   

                                     stillingsbeskrivelse  \
28849   # ligger sentralt til på Skullerud og har ca. ...   
86773   Arbeidssted: Kommune i Møre og Romsdal fylke, ...   
79955   Bømlo brannvern har ledig 100 % vikariat fram ...   
100626  Akuttmottak er organisatorisk tilknyttet kirur...   
85658   For kunde søkjer vi erfarne og sjølvgående tøm...   
114752  Vi søker en tydelig og handlingsdyktig rådmann...   
49825   Killingrud ungdomsskole har ledig vikariater, ...   
86183   Vi ønsker å ansette en person som i første omg...  

In [8]:
# Make the pipeline that transforms our data and trains the classifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_df=.25)),
    ('svc', LinearSVC()),
])

pipeline.fit(X_train['stillingsbeskrivelse'].values, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.25, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tr...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [9]:
# Test the data by running the test data set on the trained pipeline
predicted = pipeline.predict(X_test['stillingsbeskrivelse'].values)
pprint(predicted)
np.mean(predicted == y_test)

array([u'Helse, pleie og omsorg', u'Meglere og konsulenter',
       u'Akademiske yrker', ..., u'Serviceyrker og annet arbeid',
       u'Undervisning', u'Undervisning'], dtype=object)


0.7523285351397121

In [10]:
from sklearn import metrics

# Print some stats of our results
print(metrics.classification_report(y_test, predicted))
print(metrics.accuracy_score(y_test, predicted))

                                    precision    recall  f1-score   support

                  Akademiske yrker       0.67      0.62      0.64       195
           Barne- og ungdomsarbeid       0.77      0.60      0.67        84
            Butikk- og salgsarbeid       0.82      0.85      0.83       171
                    Bygg og anlegg       0.69      0.68      0.69       114
            Helse, pleie og omsorg       0.84      0.94      0.89       670
                    Industriarbeid       0.50      0.49      0.50        81
Ingen yrkesbakgrunn eller uoppgitt       0.00      0.00      0.00        16
              Ingeniør- og ikt-fag       0.62      0.74      0.68       164
       Jordbruk, skogbruk og fiske       0.67      0.25      0.36         8
                      Kontorarbeid       0.74      0.56      0.64        97
                            Ledere       0.38      0.25      0.30        61
            Meglere og konsulenter       0.59      0.53      0.56        89
           

/opt/conda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
# Try the pipeline with some custom data

prediction = pipeline.predict(['difi leikanger'])
pprint(prediction)

array([u'Ingeni\xf8r- og ikt-fag'], dtype=object)


In [23]:
# Analyze which words are most characteristic of each category

tfidf = TfidfVectorizer()

df_sample = df.sample(n=100) # Will take very much time and memory if n>100000

features = tfidf.fit_transform(df_sample['stillingsbeskrivelse'])
labels = df_sample['yrke_grovgruppe']
features.shape

from sklearn.feature_selection import chi2
N = 10
for label in labels.drop_duplicates():
    features_chi2 = chi2(features, labels == label)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    print(u"# '{}':".format(unicode(label)))
    print("  . Most correlated unigrams:\n. {}".format(u'\n. '.join(unigrams[-N:]).encode('utf-8')))


# 'Bygg og anlegg':
  . Most correlated unigrams:
. reisejobb
. ordretilgang
. 28
. gravemaskinfører
. grunn
. molde
. tømrer
. anlegg
. vedlikehald
. rørlegger
# 'Helse, pleie og omsorg':
  . Most correlated unigrams:
. and
. deg
. as
. 75
. behandling
. 09
. id
. sjukepleiar
. pasienter
. sykepleier
# 'Serviceyrker og annet arbeid':
  . Most correlated unigrams:
. ringerike
. finn
. sko
. åmli
. mm
. relativt
. storsenter
. politistasjon
. politidistrikt
. 00
# 'Kontorarbeid':
  . Most correlated unigrams:
. hat
. fakturering
. administrasjons
. skjema
. sor
. soraurdal
. planavdelingen
. aurdal
. søknadssenter
. mammut
# 'Butikk- og salgsarbeid':
  . Most correlated unigrams:
. kunder
. flesland
. skranke
. produkter
. salgs
. markedssjef
. medieselger
. kundebehandling
. distriktssjef
. salg
# 'Undervisning':
  . Most correlated unigrams:
. spesialpedagogisk
. the
. mortensnes
. nordland
. of
. ungdomsarbeid
. lærar
. barnehage
. elever
. skole
# 'Ingeniør- og ikt-fag':
  . Most co